# 실습 3 토트넘 선수들의 등번호, 이름, 국적, 이미지를 불러오기   
선수 이미지는 player_images에 저장하고, 선수 등번호, 이름, 국적은 players.csv에 저장한다.

## 라이브러리 불러오기

In [1]:
import requests
from bs4 import BeautifulSoup
from io import BytesIO
from PIL import Image
import os
import re
import pandas as pd

print("1단계 완료: 라이브러리 불러오기 성공")

1단계 완료: 라이브러리 불러오기 성공


## 웹페이지 연결 확인 및 폴더 생성

In [2]:
# 크롤링할 페이지 URL
url = "https://www.tottenhamhotspur.com/teams/men/players/"  # 실제 선수 정보 페이지 URL로 교체

# HTML 페이지 불러오기
response = requests.get(url)

# 이미지 저장 폴더 생성
image_dir = "player_images"
if not os.path.exists(image_dir):
    os.makedirs(image_dir)

# 정상 연결 확인
if response.status_code == 200:
    print("2단계 완료: 페이지 불러오기 성공")
else:
    print("페이지 연결 실패:", response.status_code)

2단계 완료: 페이지 불러오기 성공


번호   
#react_YBEVfG5bnUyUOjUrKS2eA > div > div:nth-child(1) > a:nth-child(2) > div.PlayersPlayer__info > div.PlayersPlayer__number

이름   
#react_YBEVfG5bnUyUOjUrKS2eA > div > div:nth-child(1) > a:nth-child(2) > div.PlayersPlayer__info > div.PlayersPlayer__name

나라   
#react_YBEVfG5bnUyUOjUrKS2eA > div > div:nth-child(1) > a:nth-child(8) > div.PlayersPlayer__info > div.PlayersPlayer__country

이미지   
#react_YBEVfG5bnUyUOjUrKS2eA > div > div:nth-child(1) > a:nth-child(2) > div.PlayersPlayer__photo > img

## HTML 파싱 및 크롤링

In [ ]:
# HTML 파싱
soup = BeautifulSoup(response.content, "html.parser")

# HTML 구조에 맞는 선택자 작성 (국적 추가)
number_list = soup.select("div.PlayersPlayer__info > div.PlayersPlayer__number")  # 선수 번호
name_list = soup.select("div.PlayersPlayer__info > div.PlayersPlayer__name")      # 선수 이름
image_list = soup.select("div.PlayersPlayer__photo > img")                      # 선수 이미지 태그
country_list = soup.select("div.PlayersPlayer__info > div.PlayersPlayer__country") # 선수 국적

print(f"3단계 완료: 수집한 선수 수 = {len(name_list)}")

# 파일명에서 사용할 수 없는 문자 제거
pattern = r'[\\/:"*?<>|]'
# image_dir 변수가 정의되어 있다고 가정합니다. 예: image_dir = "player_images"
# os.makedirs(image_dir, exist_ok=True)

results = []
# zip에 country_list를 추가하여 국적 정보도 함께 순회합니다.
for number, name, image, country in zip(number_list, name_list, image_list, country_list):
    # 1. 이미지 다운로드 및 저장 (이 부분은 변경 없음)
    image_url = image.get("data-src")
    if image_url:
        try:
            img_response = requests.get(image_url)
            img = Image.open(BytesIO(img_response.content))
            filename = f"{number.text.strip()}_{name.text.strip()}"
            filename = re.sub(pattern, "", filename)
            # image_dir 변수에 이미지 저장
            img.save(os.path.join(image_dir, filename + ".png"))
        except Exception as e:
            print(f"이미지 처리 중 오류 발생: {filename}.png - {e}")

    # 2. CSV 파일에 저장할 데이터만 results 리스트에 추가
    # (번호, 이름, 국적)
    results.append([
        number.text.strip(),
        name.text.strip(),
        country.text.strip() # 국적 텍스트 추출
    ])

print("3단계 완료: 이미지 다운로드 및 데이터 수집 성공")

3단계 완료: 수집한 선수 수 = 34
3단계 완료: 이미지 다운로드 및 데이터 수집 성공


## CSV 저장

In [6]:
# DataFrame으로 변환 후 CSV 저장
df = pd.DataFrame(results, columns=["번호", "이름", "국적"])
df.to_csv("players.csv", index=False, encoding="utf-8-sig")

print("4단계 완료: players.csv 저장 완료!")

4단계 완료: players.csv 저장 완료!
